# py data easyquotation

In [ ]:
from datetime import *
import time
import easyquotation
import pymysql
from dbutils.pooled_db import PooledDB

mysqlInfo = {
    "host": "localhost",
    "user": "root",
    "passwd": "password",
    "db": "python",
    "port": 3306,
    "charset": "utf8",
}


class ConnMysql(object):
    __pool = None

    def __init__(self):
        # 构造函数，创建数据库连接、游标
        self.coon = ConnMysql.getmysqlconn()
        self.cur = self.coon.cursor(cursor=pymysql.cursors.DictCursor)

    # 数据库连接池连接
    @staticmethod
    def getmysqlconn():
        global __pool
        if ConnMysql.__pool is None:
            __pool = PooledDB(
                creator=pymysql,
                mincached=2,
                maxcached=5,
                maxconnections=6,
                maxshared=3,
                blocking=True,
                maxusage=None,
                setsession=[],
                ping=0,
                host=mysqlInfo["host"],
                user=mysqlInfo["user"],
                passwd=mysqlInfo["passwd"],
                db=mysqlInfo["db"],
                port=mysqlInfo["port"],
                charset=mysqlInfo["charset"],
            )
        return __pool.connection()

    # 插入、修改、删除一条
    def sql_change_msg(self, sql):
        change_sql = self.cur.execute(sql)
        self.coon.commit()
        return change_sql

    # 查询多条
    def sql_select_many(self, sql, count=None):
        print(self.coon)
        self.cur.execute(sql)
        if count is None:
            select_res = self.cur.fetchall()
        else:
            select_res = self.cur.fetchmany(count)
        return select_res

    # 释放资源
    def release(self):
        self.coon.close()
        self.cur.close()


db = ConnMysql()

# 获取数据存入数据库
def getData():
    quotation = easyquotation.use("tencent")  # 新浪 ['sina'] 腾讯 ['tencent', 'qq']
    while True:
        if (
            datetime.now().strftime("%H:%M") > "11:30"
            and datetime.now().strftime("%H:%M") < "13:00"
            or datetime.now().strftime("%H:%M") > "15:00"
        ):
            print("收市了")
            break
        try:
            all_data = quotation.market_snapshot(prefix=True)
            for val in all_data.values():
                db.sql_change_msg(
                    "insert into data_list(name,code,now,close,open,volume,bid_volume,ask_volume,bid1,bid1_volume,bid2,bid2_volume,bid3,bid3_volume,bid4,bid4_volume,bid5,bid5_volume,ask1,ask1_volume,ask2,ask2_volume,ask3,ask3_volume,ask4,ask4_volume,ask5,ask5_volume,最近逐笔成交,datetime,涨跌,`涨跌(%)`,high,low,`价格/成交量(手)/成交额`,`成交量(手)`,`成交额(万)`,turnover,PE,unknown,high_2,low_2,振幅,流通市值,总市值,PB,涨停价,跌停价,量比,委差,均价,`市盈(动)`,`市盈(静)`) values('"
                    + str(val["name"])
                    + "','"
                    + str(val["code"])
                    + "','"
                    + str(val["now"])
                    + "','"
                    + str(val["close"])
                    + "','"
                    + str(val["open"])
                    + "','"
                    + str(val["volume"])
                    + "','"
                    + str(val["bid_volume"])
                    + "','"
                    + str(val["ask_volume"])
                    + "','"
                    + str(val["bid1"])
                    + "','"
                    + str(val["bid1_volume"])
                    + "','"
                    + str(val["bid2"])
                    + "','"
                    + str(val["bid2_volume"])
                    + "','"
                    + str(val["bid3"])
                    + "','"
                    + str(val["bid3_volume"])
                    + "','"
                    + str(val["bid4"])
                    + "','"
                    + str(val["bid4_volume"])
                    + "','"
                    + str(val["bid5"])
                    + "','"
                    + str(val["bid5_volume"])
                    + "','"
                    + str(val["ask1"])
                    + "','"
                    + str(val["ask1_volume"])
                    + "','"
                    + str(val["ask2"])
                    + "','"
                    + str(val["ask2_volume"])
                    + "','"
                    + str(val["ask3"])
                    + "','"
                    + str(val["ask3_volume"])
                    + "','"
                    + str(val["ask4"])
                    + "','"
                    + str(val["ask4_volume"])
                    + "','"
                    + str(val["ask5"])
                    + "','"
                    + str(val["ask5_volume"])
                    + "','"
                    + str(val["最近逐笔成交"])
                    + "','"
                    + str(val["datetime"])
                    + "','"
                    + str(val["涨跌"])
                    + "','"
                    + str(val["涨跌(%)"])
                    + "','"
                    + str(val["high"])
                    + "','"
                    + str(val["low"])
                    + "','"
                    + str(val["价格/成交量(手)/成交额"])
                    + "','"
                    + str(val["成交量(手)"])
                    + "','"
                    + str(val["成交额(万)"])
                    + "','"
                    + str(val["turnover"])
                    + "','"
                    + str(val["PE"])
                    + "','"
                    + str(val["unknown"])
                    + "','"
                    + str(val["high_2"])
                    + "','"
                    + str(val["low_2"])
                    + "','"
                    + str(val["振幅"])
                    + "','"
                    + str(val["流通市值"])
                    + "','"
                    + str(val["总市值"])
                    + "','"
                    + str(val["PB"])
                    + "','"
                    + str(val["涨停价"])
                    + "','"
                    + str(val["跌停价"])
                    + "','"
                    + str(val["量比"])
                    + "','"
                    + str(val["委差"])
                    + "','"
                    + str(val["均价"])
                    + "','"
                    + str(val["市盈(动)"])
                    + "','"
                    + str(val["市盈(静)"])
                    + "')"
                )
                #以上可以挑选自己需要的字段，我这里是把所有的字段都存入了数据库
        except Exception as e:
            print("连接有问题: ", e, datetime.now())
            quotation = easyquotation.use("sina")  
        time.sleep(4)


while True:
    nowTime = datetime.now().strftime("%H:%M")
    day = datetime.now().isoweekday()  # 获取当前星期几 
    if day != 6 and day != 7:
        if nowTime >= "09:30" and nowTime < "11:30":
            print("开盘了!", datetime.now())
            getData()
        elif nowTime >= "13:00" and nowTime < "15:00":
            print("开盘了!", datetime.now())
            getData()
        elif nowTime > "15:00":
            break
        else:
            print("收市了!", datetime.now())
    else:
        print("收市了!", datetime.now())
        break
    time.sleep(20)
    db.release()
